In [1]:


GOOD = 'v1.0.4'  # where script exits 0
BAD = 'v1.5.3'  # where script exists 1
SCRIPT = """
import pandas as pd


    n = 2 ** i

    print(f"Trying n = 2 ** {i} '{n}'...")

    df = pd.DataFrame([['A', 14], ['A', n]], columns=['gb', 'val'])

    gb_sum = df.groupby('gb').sum().values[0][0]
    df_sum = df.sum().values[1]
    
    assert gb_sum != df_sum

    del df
"""

In [2]:
import os
import subprocess
import shlex
import shutil

if os.path.exists('pandas-dev'):
    shutil.rmtree('pandas-dev')
_ = !git clone https://github.com/pandas-dev/pandas.git pandas-dev

In [3]:
with open('pandas-dev/t.py', 'w') as fd:
    fd.write(SCRIPT)
    
VERSIONEER = """
import os
has_versioneer = False
with open('requirements-dev.txt') as fd:
    for line in fd:
        if 'versioneer' in line:
            has_versioneer = True
            cython_rqt = line
print('has versioneer: ', has_versioneer)
os.system(f'pip install {line}')
if has_versioneer:
    os.system('pip install versioneer[toml]')
else:
    os.system('pip uninstall versioneer -y')
"""
    
with open('pandas-dev/install_versioneer.py', 'w') as fd:
    fd.write(VERSIONEER)

BISECT = """
python install_versioneer.py
python setup.py build_ext -j 1
pip install -e . --no-build-isolation --no-use-pep517
python t.py
"""

with open('pandas-dev/bisect.sh', 'w') as fd:
    fd.write(BISECT)
    
SETUP_BISECT = f"""
git bisect start
git bisect good {GOOD}
git bisect bad {BAD}
"""

with open('pandas-dev/setup_bisect.sh', 'w') as fd:
    fd.write(SETUP_BISECT)
    
CHECKOUT_BAD = f"""
git checkout {BAD}
"""

with open('pandas-dev/checkout_bad.sh', 'w') as fd:
    fd.write(CHECKOUT_BAD)

CHECKOUT_GOOD = f"""
git checkout {GOOD}
"""

with open('pandas-dev/checkout_good.sh', 'w') as fd:
    fd.write(CHECKOUT_GOOD)

In [4]:
CHECKOUT_BAD

'\ngit checkout v1.5.3\n'

In [5]:
!cd pandas-dev && bash checkout_bad.sh

Note: switching to 'v1.5.3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2e218d1098 RLS: 1.5.3


In [6]:
!cd pandas-dev && git log -n 1

commit 2e218d10984e9919f0296931d92ea851c6a6faf5 (HEAD, tag: v1.5.3)
Author: Pandas Development Team <pandas-dev@python.org>
Date:   Thu Jan 19 10:26:43 2023 +0700

    RLS: 1.5.3


In [7]:
!cd pandas-dev && pip install cython numpy python-dateutil pytz scipy versioneer[toml]

  Using cached Cython-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.5 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.1 MB)
  Using cached versioneer-0.29-py3-none-any.whl (46 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [8]:
_ = !cd pandas-dev && python setup.py build_ext -j 1

In [9]:
_

['Compiling pandas/_libs/algos.pyx because it changed.',
 'Compiling pandas/_libs/arrays.pyx because it changed.',
 'Compiling pandas/_libs/groupby.pyx because it changed.',
 'Compiling pandas/_libs/hashing.pyx because it changed.',
 'Compiling pandas/_libs/hashtable.pyx because it changed.',
 'Compiling pandas/_libs/index.pyx because it changed.',
 'Compiling pandas/_libs/indexing.pyx because it changed.',
 'Compiling pandas/_libs/internals.pyx because it changed.',
 'Compiling pandas/_libs/interval.pyx because it changed.',
 'Compiling pandas/_libs/join.pyx because it changed.',
 'Compiling pandas/_libs/lib.pyx because it changed.',
 'Compiling pandas/_libs/missing.pyx because it changed.',
 'Compiling pandas/_libs/parsers.pyx because it changed.',
 'Compiling pandas/_libs/reduction.pyx because it changed.',
 'Compiling pandas/_libs/ops.pyx because it changed.',
 'Compiling pandas/_libs/ops_dispatch.pyx because it changed.',
 'Compiling pandas/_libs/properties.pyx because it changed.

In [10]:
print("done")

done


In [11]:
!cd pandas-dev && pip install -e . --no-build-isolation --no-use-pep517

Obtaining file:///home/dan/notebooks/pandas-dev/pandas-dev
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [100 lines of output]
      Compiling pandas/_libs/tslibs/timedeltas.pyx because it changed.
      Compiling pandas/_libs/window/aggregations.pyx because it changed.
      [1/2] Cythonizing pandas/_libs/tslibs/timedeltas.pyx
      [2/2] Cythonizing pandas/_libs/window/aggregations.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
              if ((len(number) or len(frac)) and not len(unit)
                      and current_unit is None):
                  raise ValueError("no units specified")
      
              if len(frac) > 0 and len(frac) <= 3:
                  m = 10**(3 -len(frac)) * 1000 * 1000
                                                ^
      -------------------------------------

In [12]:
!cd pandas-dev && python -c 'import pandas; print(pandas.__version__)'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/__init__.py", line 22, in <module>
    from pandas.compat import is_numpy_dev as _is_numpy_dev  # pyright: ignore # noqa:F401
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/compat/__init__.py", line 17, in <module>
    from pandas._typing import F
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/_typing.py", line 9, in <module>
    from typing import (
ImportError: cannot import name 'Literal' from 'typing' (/usr/lib/python3.7/typing.py)


In [13]:
!cd pandas-dev && bash bisect.sh

has versioneer:  False

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Found existing installation: versioneer 0.29
Uninstalling versioneer-0.29:
  Successfully uninstalled versioneer-0.29
Compiling pandas/_libs/tslibs/timedeltas.pyx because it changed.
Compiling pandas/_libs/window/aggregations.pyx because it changed.
[1/2] Cythonizing pandas/_libs/tslibs/timedeltas.pyx

Error compiling Cython file:
------------------------------------------------------------
...
        if ((len(number) or len(frac)) and not len(unit)
                and current_unit is None):
            raise ValueError("no units specified")

        if len(frac) > 0 and len(frac) <= 3:
            m = 10**(3 -len(frac)) * 1000 * 1000
                                          ^
------------------------------------------------------------

pandas/_libs/tslibs/timedeltas.pyx:611:42: Cannot assign type 'double' to 'int64_t'

Error compiling Cython file:


Traceback (most recent call last):
  File "t.py", line 2, in <module>
    import pandas as pd
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/__init__.py", line 22, in <module>
    from pandas.compat import is_numpy_dev as _is_numpy_dev  # pyright: ignore # noqa:F401
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/compat/__init__.py", line 17, in <module>
    from pandas._typing import F
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/_typing.py", line 9, in <module>
    from typing import (
ImportError: cannot import name 'Literal' from 'typing' (/usr/lib/python3.7/typing.py)


In [14]:
!cd pandas-dev && bash checkout_good.sh

Previous HEAD position was 2e218d1098 RLS: 1.5.3
HEAD is now at 1ce1c3c1ef RLS: 1.0.4


In [15]:
!cd pandas-dev && bash bisect.sh

has versioneer:  False
  Cloning https://github.com/pandas-dev/pydata-sphinx-theme.git (to revision master) to /tmp/pip-req-build-muqq300t
  Running command git clone --filter=blob:none --quiet https://github.com/pandas-dev/pydata-sphinx-theme.git /tmp/pip-req-build-muqq300t
  Running command git checkout -q master
  error: pathspec 'master' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q master did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q master did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Compiling pandas/_libs/algos.pyx because it changed.
Compili

[ 2/40] Cythonizing pandas/_libs/groupby.pyx
[ 3/40] Cythonizing pandas/_libs/hashing.pyx
[ 4/40] Cythonizing pandas/_libs/hashtable.pyx
[ 5/40] Cythonizing pandas/_libs/index.pyx
[ 6/40] Cythonizing pandas/_libs/indexing.pyx
[ 7/40] Cythonizing pandas/_libs/internals.pyx
[ 8/40] Cythonizing pandas/_libs/interval.pyx
[ 9/40] Cythonizing pandas/_libs/join.pyx
[10/40] Cythonizing pandas/_libs/lib.pyx
[11/40] Cythonizing pandas/_libs/missing.pyx
[12/40] Cythonizing pandas/_libs/ops.pyx
[13/40] Cythonizing pandas/_libs/ops_dispatch.pyx
[14/40] Cythonizing pandas/_libs/parsers.pyx
[15/40] Cythonizing pandas/_libs/properties.pyx
[16/40] Cythonizing pandas/_libs/reduction.pyx

Error compiling Cython file:
------------------------------------------------------------
...
            object cached_typ = None

        arr = self.arr
        chunk = self.dummy
        dummy_buf = chunk.data
        chunk.data = arr.data
             ^
------------------------------------------------------------

p

Traceback (most recent call last):
  File "/home/dan/venv/pandas-bisect/lib/python3.7/site-packages/Cython/Build/Dependencies.py", line 1325, in cythonize_one_helper
    return cythonize_one(*m)
  File "/home/dan/venv/pandas-bisect/lib/python3.7/site-packages/Cython/Build/Dependencies.py", line 1301, in cythonize_one
    raise CompileError(None, pyx_file)
Cython.Compiler.Errors.CompileError: pandas/_libs/tslibs/timedeltas.pyx
[34/40] Cythonizing pandas/_libs/tslibs/timestamps.pyx
[35/40] Cythonizing pandas/_libs/tslibs/timezones.pyx
[36/40] Cythonizing pandas/_libs/tslibs/tzconversion.pyx
[37/40] Cythonizing pandas/_libs/window/aggregations.pyx

Error compiling Cython file:
------------------------------------------------------------
...
        buf = <float64_t *>arr.data
        bufarr = np.empty(win, dtype=float)
        oldbuf = <float64_t *>bufarr.data
        for i in range((win - offset), (N - offset)):
            buf = buf + 1
            bufarr.data = <char *>buf
            

Traceback (most recent call last):
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/__init__.py", line 32, in <module>
    from pandas._libs import hashtable as _hashtable, lib as _lib, tslib as _tslib
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/_libs/__init__.py", line 3, in <module>
    from .tslibs import (
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/_libs/tslibs/__init__.py", line 3, in <module>
    from .conversion import localize_pydatetime, normalize_date
ModuleNotFoundError: No module named 'pandas._libs.tslibs.conversion'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "t.py", line 2, in <module>
    import pandas as pd
  File "/home/dan/notebooks/pandas-dev/pandas-dev/pandas/__init__.py", line 37, in <module>
    f"C extension: {module} not built. If you want to import "
ImportError: C extension: No module named 'pandas._libs.tslibs.conversion' not built. If you want to import pa

In [16]:
!cd pandas-dev && bash setup_bisect.sh

Bisecting: a merge base must be tested
[d3f08566a80239a18a813ebda9a2ebb0368b1dc5] RLS: 1.0.0rc0


In [17]:
output = !cd pandas-dev && git bisect run bash bisect.sh

In [18]:
for line in output:
    if 'is the first bad commit' in line:
        print(line)
        commit = line.split()[0]

In [19]:
os.system(f'cd pandas-dev && git log --format=%B -n 1 {commit}')

NameError: name 'commit' is not defined

In [ ]:
with open('output.txt', 'w') as fd:
    fd.write('\n'.join(output))

In [ ]:
output

In [ ]:
GOOD

In [ ]:
BAD